##구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
IMAGE_width=192
IMAGE_height=144
IMAGE_size=(IMAGE_width,IMAGE_height)
IMAGE_channels=3

## 파일 경로 지정

In [4]:
train_path='/gdrive/My Drive/생체인증보안/홍채/train_set'
test_path='/gdrive/My Drive/생체인증보안/홍채/test_set'

## 라이브러리 import

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
import glob, os
import cv2
from PIL import Image

## Augmentation 후 저장

In [ ]:
# 랜덤시드 고정시키기
np.random.seed(15)

from keras.preprocessing.image import ImageDataGenerator, img_to_array

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(brightness_range=[0.8,1.2],
                                  zoom_range=[0.9, 1.1],
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  fill_mode='constant')
 

In [ ]:
%cd '/gdrive/My Drive/생체인증보안/홍채/train_set'
nowpath='/gdrive/My Drive/생체인증보안/홍채/train_set'

for filename in os.listdir(nowpath):
  img=Image.open(filename)
  img = img_to_array(img)
  img = img.reshape((1,) + img.shape)
  
  i = 0
  # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
  for batch in data_aug_gen.flow(img, batch_size=1, save_to_dir='./', save_prefix=filename, save_format='png'):
     i += 1
     if i >4: 
         break

/gdrive/My Drive/생체인증보안/홍채/train_set


In [ ]:
images=sorted(glob.glob("*"))

print(len(images))

767


##데이터 불러오기

In [ ]:
x_data=[]
y_data=[]

file_names = os.listdir(train_path)
for file_name in file_names:
    file_path='{}/{}'.format(train_path, file_name)
    image=Image.open(file_path)
    image=image.resize((IMAGE_width,IMAGE_height))
    np_image=np.asarray(image)
    np_image=np_image/255

    name=file_name[0:3]
    y_name=int(name.lstrip("0"))
    x_data.append(np_image)
    y_data.append(y_name-1)


In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [ ]:
x_data.shape

(767, 144, 192, 3)

In [ ]:
y_data.shape

(767,)

In [ ]:
np.save("/gdrive/My Drive/생체인증보안/홍채/x_data.npy",x_data)
np.save("/gdrive/My Drive/생체인증보안/홍채/y_data.npy",y_data)

In [6]:
x_data = np.load('/gdrive/My Drive/생체인증보안/홍채/x_data.npy')
y_data = np.load('/gdrive/My Drive/생체인증보안/홍채/y_data.npy')

##모델링

In [ ]:
input = tf.keras.layers.Input(shape=(IMAGE_width, IMAGE_height, IMAGE_channels))
tf.keras.backend.is_keras_tensor(input)

True

In [23]:
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201, DenseNet121
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import losses, optimizers, activations, metrics

base_model=DenseNet121(input_shape=(IMAGE_height,IMAGE_width,IMAGE_channels),include_top=False)

x = base_model.output
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)

predictions = Dense(64, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

adam=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 144, 192, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 150, 198, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 72, 96, 64)   9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 72, 96, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

##Callback

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#조기종료
earlystop=EarlyStopping(monitor='sparse_categorical_accuracy',patience=10)

#학습율 조정
learning_rate_reduction=ReduceLROnPlateau(monitor='sparse_categorical_accuracy',
                                          patience=2,
                                          verbose=1,
                                          factor=0.5,
                                          min_lr=0.000001)
#callback 설정
callback=[earlystop,learning_rate_reduction]

##모델 학습(Cross-Validation)

In [24]:
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=3)
cvscores = []

for train_idx,val_idx in kfold.split(x_data,y_data):
  x_train,x_val=x_data[train_idx],x_data[val_idx]
  y_train,y_val=y_data[train_idx],y_data[val_idx]

  model.fit(x_train,y_train,epochs=10,batch_size=8,verbose=1,callbacks=callback)
  
  scores = model.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/10
64/64 [==============================] - 13s 74ms/step - loss: 3.9004 - sparse_categorical_accuracy: 0.3112
Epoch 2/10
64/64 [==============================] - 5s 74ms/step - loss: 0.3451 - sparse_categorical_accuracy: 0.8982
Epoch 3/10
64/64 [==============================] - 5s 74ms/step - loss: 0.1135 - sparse_categorical_accuracy: 0.9706
Epoch 4/10
64/64 [==============================] - 5s 74ms/step - loss: 0.0350 - sparse_categorical_accuracy: 0.9961
Epoch 5/10
64/64 [==============================] - 5s 73ms/step - loss: 0.0125 - sparse_categorical_accuracy: 1.0000
Epoch 6/10
64/64 [==============================] - 5s 73ms/step - loss: 0.0243 - sparse_categorical_accuracy: 0.9902
Epoch 7/10
64/64 [==============================] - 5s 74ms/step - loss: 0.0424 - sparse_categorical_accuracy: 0.9843

Epoch 00007: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 8/10
64/64 [==============================] - 5s 74ms/step - loss: 0.0547 - sparse_cat

In [15]:
print("평균 검증 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 검증 Accuracy: 92.19% (+/- 11.05%)


In [31]:
%cd '/gdrive/My Drive/생체인증보안/홍채/'
model.save('iris_rec2.h5')

/gdrive/My Drive/생체인증보안/홍채


##모델 성능 지표</br>
학습 데이터 중 split한 validation 데이터 사용

In [32]:
y_train_pred=model.predict(x_train)
y_train_pred

array([[1.01691322e-08, 4.50264039e-08, 2.41360310e-09, ...,
        4.70293715e-09, 1.74685070e-07, 1.95615413e-09],
       [1.79042758e-07, 1.39035565e-08, 5.64463187e-09, ...,
        1.64618967e-07, 5.48609798e-08, 8.78716762e-08],
       [2.25898730e-10, 9.99999404e-01, 3.23959054e-11, ...,
        6.11086112e-11, 5.87886850e-10, 1.17339666e-10],
       ...,
       [3.22407789e-09, 3.00908715e-10, 8.37095655e-08, ...,
        2.15987498e-07, 1.46278660e-08, 8.94707242e-09],
       [3.69774433e-10, 2.46598262e-07, 5.36939104e-10, ...,
        7.72711672e-10, 1.15272147e-09, 1.66750530e-10],
       [1.79433499e-11, 9.69239466e-10, 3.71308695e-09, ...,
        2.43772169e-10, 2.14439222e-09, 2.26762463e-11]], dtype=float32)

In [33]:
print(y_train)

[ 3  4  1  3  2  0  2  0  1  6 54 27 13  7 53 24 60  8 52 27 20 30 61 40
 26 46 38 40 49 41 58 53 16 39 50 26 30 11 54 33 35 57 58 62 11 59 21 32
  4  8 47  5 21 35 38 22 12 20 25 50 48 44  7 45 36 18 55 31 52 10 13 32
 14 28 47 48 14 43 15 37 25 31 18 55 45 42 23 41 29 59  6 63 10 51 42 49
 12 29  9  5 17 33 34 34 19 17 56 22 61 37 43 56 23 60 36 28 62 57 51 39
 63 16 24 15 19 44  9 46  3  3  3  3  3  4  4  4  4  4  1  1  1  1  1  3
  2  2  2  2  2  0  0  0  0  0  2  0  1  6  6  6  6  6 54 54 54 54 54 27
 27 27 27 27 13 13 13 13 13  7  7  7  7  7 53 53 53 53 53 24 24 24 24 24
 60 60 60 60 60  8  8  8  8  8 52 52 52 52 52 27 20 20 20 20 20 30 30 30
 30 30 61 61 61 61 61 40 40 40 40 40 26 26 26 26 26 46 46 46 46 46 38 38
 38 38 38 40 49 49 49 49 49 41 41 41 41 41 58 58 58 58 58 53 16 16 16 16
 16 39 39 39 39 39 50 50 50 50 50 26 30 11 11 11 11 11 54 33 33 33 33 33
 35 35 35 35 35 57 57 57 57 57 58 62 62 62 62 62 11 59 59 59 59 59 21 21
 21 21 21 32 32 32 32 32  4  8 47 47 47 47  5  5  5

In [34]:
y_pred_train=[]
for i in range(len(y_train)):
  n=np.argmax(y_train_pred[i])
  y_pred_train.append(n)

In [35]:
print(y_pred_train)

[3, 4, 1, 3, 2, 0, 2, 0, 1, 6, 54, 27, 13, 7, 53, 24, 60, 8, 52, 27, 20, 30, 61, 40, 26, 46, 38, 40, 49, 41, 58, 53, 16, 39, 50, 26, 30, 11, 54, 33, 35, 57, 58, 62, 11, 59, 21, 32, 4, 8, 47, 5, 21, 35, 38, 22, 12, 20, 25, 50, 48, 44, 7, 45, 36, 18, 55, 31, 52, 10, 13, 32, 14, 28, 47, 48, 14, 43, 15, 37, 25, 31, 18, 55, 45, 42, 23, 41, 29, 59, 6, 63, 10, 51, 42, 49, 12, 29, 9, 5, 17, 33, 34, 34, 19, 17, 56, 22, 61, 37, 43, 56, 23, 60, 36, 28, 62, 57, 51, 39, 63, 16, 24, 15, 19, 44, 9, 46, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 3, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 6, 6, 6, 6, 6, 54, 54, 54, 54, 54, 27, 27, 27, 27, 27, 13, 13, 13, 13, 13, 7, 7, 7, 7, 7, 53, 53, 53, 53, 53, 24, 24, 24, 24, 24, 60, 60, 60, 60, 60, 8, 8, 8, 8, 8, 52, 52, 52, 52, 52, 27, 20, 20, 20, 20, 20, 30, 30, 30, 30, 30, 61, 61, 61, 61, 61, 40, 40, 40, 40, 40, 26, 26, 26, 26, 26, 46, 46, 46, 46, 46, 38, 38, 38, 38, 38, 40, 49, 49, 49, 49, 49, 41, 41, 41, 41, 41, 58, 58, 58, 58, 58, 53, 16, 16, 16, 16, 16, 39,

In [36]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_train,y_pred_train)                         
precision = precision_score(y_train,y_pred_train,average='micro') 
recall = recall_score(y_train,y_pred_train, average='micro')      
f1 = f1_score(y_train,y_pred_train, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


##테스트

In [16]:
import natsort
%cd '/gdrive/My Drive/생체인증보안/홍채/test_set'

fname=[]
x_test=[]
y_test=[]

for filename in os.listdir(test_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for filename in fname:
  img=Image.open(filename)
  print(filename)
  img=img.resize((IMAGE_width,IMAGE_height))
  np_image=np.asarray(img)
  np_image=np_image/255

  x_test.append(np_image)

x_test = np.array(x_test)

/gdrive/My Drive/생체인증보안/홍채/test_set
1.png
2.png
3.png
4.png
5.png
6.png
7.png
8.png
9.png
10.png
11.png
12.png
13.png
14.png
15.png
16.png
17.png
18.png
19.png
20.png
21.png
22.png
23.png
24.png
25.png
26.png
27.png
28.png
29.png
30.png
31.png
32.png
33.png
34.png
35.png
36.png
37.png
38.png
39.png
40.png
41.png
42.png
43.png
44.png
45.png
46.png
47.png
48.png
49.png
50.png
51.png
52.png
53.png
54.png
55.png
56.png
57.png
58.png
59.png
60.png
61.png
62.png
63.png
64.png
65.png
66.png
67.png
68.png
69.png
70.png
71.png
72.png
73.png
74.png
75.png
76.png
77.png
78.png
79.png
80.png
81.png
82.png
83.png
84.png
85.png
86.png
87.png
88.png
89.png
90.png
91.png
92.png
93.png
94.png
95.png
96.png
97.png
98.png
99.png
100.png
101.png
102.png
103.png
104.png
105.png
106.png
107.png
108.png
109.png
110.png
111.png
112.png
113.png
114.png
115.png
116.png
117.png
118.png
119.png
120.png
121.png
122.png
123.png
124.png
125.png
126.png
127.png
128.png
129.png
130.png
131.png
132.png
133.png
134.png


In [17]:
#x_test=x_test.reshape(256, 144, 192, 3)
x_test.shape

(256, 144, 192, 3)

In [ ]:
np.save("/gdrive/My Drive/생체인증보안/홍채/x_test.npy",x_test)

In [ ]:
x_test = np.load('/gdrive/My Drive/생체인증보안/홍채/x_test.npy')

In [25]:
y_pred=model.predict(x_test)
print(y_pred)

[[3.08167891e-09 5.55504220e-09 2.21169341e-08 ... 1.69115399e-09
  3.89925509e-10 3.82893983e-09]
 [2.52370108e-02 6.38896199e-06 2.16097351e-06 ... 1.36058312e-04
  1.93874206e-04 1.10633337e-05]
 [1.14381962e-07 4.40802432e-05 1.17587504e-07 ... 1.80966299e-06
  1.54175211e-06 3.67613239e-07]
 ...
 [2.51445044e-07 4.82863143e-05 3.19008018e-06 ... 1.01985984e-06
  1.13193664e-07 1.16583402e-03]
 [5.47873729e-04 7.80069115e-07 3.40406450e-06 ... 3.99660566e-05
  2.24745745e-04 2.29396232e-06]
 [4.29874358e-09 1.18766836e-08 2.47556708e-09 ... 1.04272843e-07
  1.27053221e-08 5.85701116e-07]]


In [26]:
result=[]
for i in range(256):
  n=np.argmax(y_pred[i])+1
  result.append(n)

In [27]:
print(result)

[46, 59, 26, 46, 51, 10, 18, 57, 14, 53, 17, 28, 12, 57, 15, 52, 9, 13, 49, 35, 4, 43, 59, 21, 4, 43, 52, 3, 22, 31, 42, 20, 22, 7, 28, 23, 53, 51, 52, 32, 25, 17, 50, 20, 63, 16, 27, 62, 40, 29, 49, 5, 41, 23, 13, 17, 22, 1, 11, 19, 53, 43, 48, 33, 19, 35, 52, 44, 8, 18, 24, 30, 11, 38, 49, 5, 61, 60, 44, 60, 20, 19, 37, 1, 30, 2, 48, 50, 10, 63, 47, 33, 3, 28, 64, 16, 12, 11, 6, 15, 34, 21, 56, 37, 40, 54, 6, 48, 37, 27, 45, 2, 10, 64, 49, 45, 32, 40, 42, 25, 9, 55, 3, 29, 30, 58, 60, 23, 62, 63, 30, 64, 28, 47, 8, 62, 41, 36, 37, 56, 10, 14, 43, 39, 50, 32, 29, 48, 55, 20, 26, 52, 61, 54, 56, 32, 21, 51, 62, 26, 36, 27, 44, 46, 8, 64, 7, 57, 47, 17, 42, 35, 19, 58, 57, 5, 13, 24, 60, 61, 63, 18, 45, 24, 61, 56, 4, 9, 7, 41, 33, 53, 38, 16, 6, 31, 34, 58, 15, 41, 31, 27, 47, 59, 54, 2, 50, 11, 25, 2, 42, 46, 13, 55, 12, 63, 55, 35, 25, 54, 52, 8, 58, 40, 18, 14, 44, 51, 22, 39, 33, 38, 38, 31, 12, 7, 36, 39, 5, 6, 34, 24, 23, 50, 34, 39, 9, 15, 4, 36, 29, 21, 14, 26, 45, 16]


In [28]:
list_df = pd.DataFrame({'Image':fname,'Answer':result})
list_df.head(20)

,Image,Answer
0,1.png,46
1,2.png,59
2,3.png,26
3,4.png,46
4,5.png,51
5,6.png,10
6,7.png,18
7,8.png,57
8,9.png,14
9,10.png,53


In [29]:
list_df.to_csv("/gdrive/My Drive/생체인증보안/홍채/result_2_4.csv")